In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

URL="https://www.redbus.in/travels/nbstc"
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  

def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME,'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  
              
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  
            
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  
            
            bus_name_elements = driver.find_elements(By.XPATH,"//div[@class='travels lh-24 f-bold d-color']")
            bus_type_elements = driver.find_elements(By.XPATH,"//div[@class='bus-type f-12 m-top-16 l-color evBus']")
            departing_time_elements = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
            duration_elements = driver.find_elements(By.XPATH,"//div[@class='dur l-color lh-24']")
            arrival_elements= driver.find_elements(By.XPATH,"//div[@class='bp-time f-19 d-color disp-Inline']")
            rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            fare_elements = driver.find_elements(By.CSS_SELECTOR,"span.f-19.f-bold")
            seat_availability_elements = driver.find_elements(By.CSS_SELECTOR,".seat-left")
            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Arrival": arrival_elements[i].text,
                    "Rating": rating_elements[i].text if i < len(rating_elements) else '0',
                    "Fare": fare_elements[i].text,
                    "Seat_Available": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

all_bus_details = []

def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs ')][text()='{page}']")))
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5) 
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
           
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            
         
        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

scrape_all_pages()

df = pd.DataFrame(all_bus_details)

df.to_csv('North_bengal.csv', index=False)

driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/siliguri-to-darjeeling: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7B0086CB5+28821]
	(No symbol) [0x00007FF7AFFF3840]
	(No symbol) [0x00007FF7AFE9578A]
	(No symbol) [0x00007FF7AFEE91BE]
	(No symbol) [0x00007FF7AFEE94AC]
	(No symbol) [0x00007FF7AFF32647]
	(No symbol) [0x00007FF7AFF0F33F]
	(No symbol) [0x00007FF7AFF2F412]
	(No symbol) [0x00007FF7AFF0F0A3]
	(No symbol) [0x00007FF7AFEDA778]
	(No symbol) [0x00007FF7AFEDB8E1]
	GetHandleVerifier [0x00007FF7B03BFCAD+3408013]
	GetHandleVerifier [0x00007FF7B03D741F+3504127]
	GetHandleVerifier [0x00007FF7B03CB5FD+3455453]
	GetHandleVerifier [0x00007FF7B014BDBB+835995]
	(No symbol) [0x00007FF7AFFFEB5F]
	(No symbol) [0x00007FF7AFFFA814]
	(No symbol) [0x00007FF7AFFFA9AD]
	(No symbol) [0x00007FF7AFFEA199]
	BaseThreadInitThunk [0x00007FFDEEED7374+20]
	RtlUserThreadStart [0x00007FFDEFDFCC91+33]

Error occurred while scraping bus details for https://www.redbus.in/bus

In [4]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Arrival,Rating,Fare,Seat_Available
0,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,NBSTC-KOL-COB-E2-3:00 pm - 171781,Non AC Seater (2+3),15:00,13h 20m,04:20,3.7,450,22 Seats available
1,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,NBSTC-KOL-RAI-SLG-19:30 - 176649,AC Seater (2+2),19:30,12h 00m,07:30,3.8,800,29 Seats available
2,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,SBSTC-KOLKATA - SILIGURI - VOLVO - 20:05 (BEL ...,Volvo AC Seater (2+2),20:05,13h 00m,09:05,3.9,1210,51 Seats available
3,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,Greenline,Volvo 9600 A/C Seater/Sleeper (2+1),19:00,13h 00m,08:00,4.4,1200,21 Seats available
4,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,Easy Ride,Volvo 9600 Multi-Axle A/C Sleeper (2+1),19:50,12h 55m,08:45,4.7,1500,21 Seats available
...,...,...,...,...,...,...,...,...,...,...
401,Kolkata to Kishanganj,https://www.redbus.in/bus-tickets/kolkata-to-k...,Prarthana Bus Service,A/C Seater / Sleeper (2+1),19:30,10h 00m,05:30,4.5,800,41 Seats available
402,Kolkata to Kishanganj,https://www.redbus.in/bus-tickets/kolkata-to-k...,Maa Durga Bus Service,Non AC Seater (2+2),19:00,13h 00m,08:00,2.8,600,16 Seats available
403,Kolkata to Kishanganj,https://www.redbus.in/bus-tickets/kolkata-to-k...,Lokenath (Maa Tara),NON A/C Seater / Sleeper (2+2),15:45,13h 00m,04:45,2.6,540,6 Seats available
404,Siliguri to Barasat (West Bengal),https://www.redbus.in/bus-tickets/siliguri-to-...,NBSTC-SLG-KOL-R1-18:30 - 156410,NON A/C Seater Push Back (2+2),18:30,11h 00m,05:30,4.4,466,6 Seats available
